In [ ]:
import pandas as pd
import re
from tqdm import tqdm
import random
import nltk
# nltk.download('word_tokenize')
# nltk.download('sent_tokenize')
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize

# Data specific preprocessing

In [ ]:

data = pd.read_csv('https://www.mustafaqazi.me/mbti_1.csv')

In [ ]:
data.head()

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||...
1,ENTP,'I'm finding the lack of me in these posts ver...
2,INTP,'Good one _____ https://www.youtube.com/wat...
3,INTJ,"'Dear INTP, I enjoyed our conversation the o..."
4,ENTJ,'You're fired.|||That's another silly misconce...


In [ ]:
data['posts'][1]

"'I'm finding the lack of me in these posts very alarming.|||Sex can be boring if it's in the same position often. For example me and my girlfriend are currently in an environment where we have to creatively use cowgirl and missionary. There isn't enough...|||Giving new meaning to 'Game' theory.|||Hello *ENTP Grin*  That's all it takes. Than we converse and they do most of the flirting while I acknowledge their presence and return their words with smooth wordplay and more cheeky grins.|||This + Lack of Balance and Hand Eye Coordination.|||Real IQ test I score 127. Internet IQ tests are funny. I score 140s or higher.  Now, like the former responses of this thread I will mention that I don't believe in the IQ test. Before you banish...|||You know you're an ENTP when you vanish from a site for a year and a half, return, and find people are still commenting on your posts and liking your ideas/thoughts. You know you're an ENTP when you...|||http://img188.imageshack.us/img188/6422/6020d1f9da

In [ ]:
data['type'].unique()

array(['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP',
       'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ'],
      dtype=object)

In [ ]:
def preprocess_story(data, remove_stop_words=True):
  df = pd.DataFrame(columns=['sentence', 'category'])

  LIMIT_SENTENCES = 23

  # iterate over the dataset
  for i in tqdm(range(len(data))):
    # initalize one instance as empty
    instance = ""
    sentence_count = 0
    row = data.iloc[i]
    # split each sentence
    sentences = row['posts'].split("|||")
    for sentence in sentences:
      # check if sentence limit is reached
      if sentence_count >= LIMIT_SENTENCES:
        break
      # Step 1: Remove links
      sentence = re.sub(r'http\S+', '', sentence)
      # Step 2: Remove multiple full stops
      sentence = re.sub(r'\.+', ".", sentence)
      # Step 3: Some sentences do not have space after full stop, remove that
      sentence = sentence.replace(".", ". ")
      # Step 4: remove multiple spaces
      sentence = re.sub(' +', ' ', sentence)
      # Step 5: remove punctuations except . , ! & ? '
      sentence = re.sub("[^a-zA-Z.,!&?']", " ", sentence)
      # Step 6, if length of sentence is less than 5 dont add it
      if len(sentence.split()) >= 5:
        if instance != "":
          instance = instance + ". " + sentence
        else:
          instance = sentence
        sentence_count += 1
    # Step 7: Extra cleaning for instance
    instance = instance.strip()
    instance = re.sub(' +', ' ', instance)
    instance = re.sub(r'\.+', ".", instance)
    instance = instance.replace(". .", ".")
    instance = instance.replace(" .", ".")
    df.loc[len(df)] = [instance, row['type']]
  return df

In [ ]:
cleaned_data = preprocess_story(data)

100%|██████████| 8675/8675 [00:42<00:00, 202.12it/s]


In [ ]:
cleaned_data.head()

,sentence,category
0,enfp and intj moments sportscenter not top ten...,INFJ
1,'I'm finding the lack of me in these posts ver...,ENTP
2,"Of course, to which I say I know that's my ble...",INTP
3,"'Dear INTP, I enjoyed our conversation the oth...",INTJ
4,That's another silly misconception. That appro...,ENTJ


In [ ]:
cleaned_data['sentence'][7]

"'I tend to build up a collection of things on my desktop that i use frequently and then move them into a folder called 'Everything' from there it get sorted into type and sub type. i ike to collect odd objects, even at work. a lot of people would call it junk but i like to collect it. Old unused software? ill take that off your hands i have a bunch of old adobe. i think its quite normal, i tend to only see my friends in real life every couple of months, as said earlier some people just dont get it but the good ones do Edit i mostly mean tolerate it. where do we go when we sleep? is dreaming another form of being awake? how many more layers of this are there if any? thoughts about sleep keep me up at night Edit sometimes im too scared. i wish i was free to follow my interests as i desired i feel as though wishes are meant for impossible things. by seeing do you mean visual interpreting or seeing as in mentally understanding the concept?. i feel as though i am incapable of creating anyt

# HAN specific preprocessing





In [ ]:
import numpy as np
import pandas as pd
import os
import pickle
import spacy

from pathlib import Path
from sklearn.model_selection import train_test_split
from gensim.utils import tokenize
from fastai.text import Tokenizer, Vocab

In [ ]:
# convert category to numerical
categories = sorted(cleaned_data['category'].unique())

def category_to_idx(cat):
  return categories.index(cat)

cleaned_data['category_class'] = cleaned_data.category.apply(category_to_idx).astype("int64")

In [ ]:
cleaned_data.head()

,sentence,category,category_class
0,enfp and intj moments sportscenter not top ten...,INFJ,8
1,'I'm finding the lack of me in these posts ver...,ENTP,3
2,"Of course, to which I say I know that's my ble...",INTP,11
3,"'Dear INTP, I enjoyed our conversation the oth...",INTJ,10
4,That's another silly misconception. That appro...,ENTJ,2


In [ ]:
texts = cleaned_data.sentence.tolist()
target = cleaned_data.category_class.tolist()

In [ ]:
texts[:3]

["enfp and intj moments sportscenter not top ten plays pranks. What has been the most life changing experience in your life?. On repeat for most of today. May the PerC Experience immerse you. The last thing my INFJ friend posted on his facebook before committing suicide the next day. Rest in peace. Hello ENFJ. Sorry to hear of your distress. It's only natural for a relationship to not be perfection all the time in every moment of existence. Try to figure the hard times as times of growth, as. Prozac, wellbrutin, at least thirty minutes of moving your legs and I don't mean moving them while sitting in your same desk chair , weed in moderation maybe try edibles as a healthier alternative. Basically come up with three items you've determined that each type or whichever types you want to do would more than likely use, given each types' cognitive functions and whatnot, when left by. All things in moderation. Sims is indeed a video game, and a good one at that. Note a good one at that is som

In [ ]:
target[:3]

[8, 3, 11]

In [ ]:
tok_func = spacy.load("en_core_web_sm")
n_cpus = os.cpu_count()
bsz = 100
texts_sents = []
for doc in tok_func.pipe(texts, n_process=n_cpus, batch_size=bsz):
    sents = [str(s) for s in list(doc.sents)]
    texts_sents.append(sents)

In [ ]:
texts_sents[0] 

['enfp and intj moments sportscenter',
 'not top ten plays pranks.',
 'What has been the most life changing experience in your life?.',
 'On repeat for most of today.',
 'May the PerC Experience immerse you.',
 'The last thing my INFJ friend posted on his facebook before committing suicide the next day.',
 'Rest in peace.',
 'Hello ENFJ.',
 'Sorry to hear of your distress.',
 "It's only natural for a relationship to not be perfection all the time in every moment of existence.",
 'Try to figure the hard times as times of growth, as.',
 'Prozac, wellbrutin, at least thirty minutes of moving your legs',
 "and I don't mean moving them while sitting in your same desk chair ,",
 'weed in moderation maybe try edibles as a healthier alternative.',
 "Basically come up with three items you've determined that each type or whichever types you want to do would more than likely use, given each types' cognitive functions and whatnot, when left by.",
 'All things in moderation.',
 'Sims is indeed a vi

In [ ]:
# from nested to flat list.
all_sents = [s for sents in texts_sents for s in sents]

In [ ]:
all_sents[:10]

['enfp and intj moments sportscenter',
 'not top ten plays pranks.',
 'What has been the most life changing experience in your life?.',
 'On repeat for most of today.',
 'May the PerC Experience immerse you.',
 'The last thing my INFJ friend posted on his facebook before committing suicide the next day.',
 'Rest in peace.',
 'Hello ENFJ.',
 'Sorry to hear of your distress.',
 "It's only natural for a relationship to not be perfection all the time in every moment of existence."]

In [ ]:
# saving the lengths of the documents: 1) for padding purposes and 2) to compute consecutive ranges 
# so we can "fold" the list again
texts_length = [0] + [len(s) for s in texts_sents]
range_idx = [sum(texts_length[: i + 1]) for i in range(len(texts_length))]

In [ ]:
texts_length[:10]

[0, 41, 80, 52, 54, 57, 79, 73, 65, 55]

In [ ]:
range_idx[:10]

[0, 41, 121, 173, 227, 284, 363, 436, 501, 556]

In [ ]:
sents_tokens = Tokenizer().process_all(all_sents)


In [ ]:
sents_tokens[0]

['enfp', 'and', 'intj', 'moments', 'sportscenter']

In [ ]:
def simple_preprocess(doc, lower=False, deacc=False, min_len=2, max_len=16):
    tokens = [
        token
        for token in tokenize(doc, lower=False, deacc=deacc, errors="ignore")
        if min_len <= len(token) <= max_len and not token.startswith("_")
    ]
    return tokens


def get_texts(texts, with_preprocess=False):
    if with_preprocess:
        texts = [" ".join(simple_preprocess(s)) for s in texts]
    tokens = Tokenizer().process_all(texts)
    return tokens

In [ ]:
sents_tokens_2 = get_texts(all_sents, with_preprocess=True)


In [ ]:
sents_tokens_2[0]


['enfp', 'and', 'intj', 'moments', 'sportscenter']

### Example

In [ ]:
s = "I don't: particularly ; like data science."


In [ ]:
get_texts([s], with_preprocess=False)[0]

['i', 'do', "n't", ':', 'particularly', ';', 'like', 'data', 'science', '.']

In [ ]:
get_texts([s], with_preprocess=True)[0]


['don', 'particularly', 'like', 'data', 'science']

### End example

In [ ]:
import re
def rm_punctuation(x):
    x = re.sub(r"\.|,|:|;", " ", x)
    # or 
    # x = x.replace(".", "").replace(",","").replace(":", "").replace(";","")
    return x

In [ ]:
new_tok = Tokenizer()
new_tok.pre_rules = [rm_punctuation] + new_tok.pre_rules

In [ ]:
new_tok.process_all([s])


[['i', 'do', "n't", 'particularly', 'like', 'data', 'science']]

In [ ]:
#  saving the lengths of sentences for padding purposes
sents_length = [len(s) for s in sents_tokens]

In [ ]:
sents_length[:5]

[5, 6, 14, 8, 9]

In [ ]:
# Create Vocabulary using fastai's Vocab class
vocab = Vocab.create(sents_tokens, max_vocab=60000, min_freq=5)

In [ ]:
# 'numericalize' each sentence
sents_numz = [vocab.numericalize(s) for s in sents_tokens]

In [ ]:
sents_numz[:5]

[[179, 14, 146, 1283, 0],
 [35, 686, 1873, 2093, 8979, 9],
 [5, 46, 105, 86, 12, 101, 115, 1441, 302, 21, 51, 115, 36, 9],
 [5, 38, 2240, 27, 101, 17, 366, 9],
 [5, 209, 12, 846, 5, 302, 11374, 18, 9]]

In [ ]:
# group the sentences again into documents
texts_numz = [sents_numz[range_idx[i] : range_idx[i + 1]] for i in range(len(range_idx[:-1]))]

In [ ]:
texts_numz[0]


[[179, 14, 146, 1283, 0],
 [35, 686, 1873, 2093, 8979, 9],
 [5, 46, 105, 86, 12, 101, 115, 1441, 302, 21, 51, 115, 36, 9],
 [5, 38, 2240, 27, 101, 17, 366, 9],
 [5, 209, 12, 846, 5, 302, 11374, 18, 9],
 [5,
  12,
  255,
  126,
  22,
  6,
  142,
  162,
  581,
  38,
  151,
  1046,
  196,
  5985,
  2228,
  12,
  473,
  194,
  9],
 [5, 786, 21, 1321, 9],
 [5, 511, 6, 336, 9],
 [5, 292, 13, 435, 17, 51, 7393, 9],
 [5,
  16,
  24,
  102,
  825,
  27,
  15,
  247,
  13,
  35,
  34,
  4213,
  56,
  12,
  73,
  21,
  221,
  482,
  17,
  1445,
  9],
 [5, 171, 13, 551, 12, 218, 261, 41, 261, 17, 2487, 11, 41, 9],
 [5, 10934, 11, 0, 11, 55, 273, 6469, 1031, 17, 1081, 51, 3322],
 [14, 10, 25, 23, 177, 1081, 78, 178, 1082, 21, 51, 148, 2987, 3816, 11],
 [2101, 21, 6845, 166, 171, 19338, 41, 15, 5799, 2196, 9],
 [5,
  528,
  228,
  75,
  29,
  553,
  3379,
  18,
  64,
  2784,
  19,
  334,
  109,
  47,
  6846,
  235,
  18,
  98,
  13,
  25,
  57,
  61,
  110,
  427,
  232,
  11,
  645,
  334,
  235,
 

In [ ]:
print(sents_tokens[1])
print([vocab.itos[i] for i in texts_numz[0][1]])

['not', 'top', 'ten', 'plays', 'pranks', '.']
['not', 'top', 'ten', 'plays', 'pranks', '.']


In [ ]:
q=0.8
maxlen_sent = int(np.quantile(sents_length, q=q))
maxlen_doc  = int(np.quantile(texts_length[1:], q=q))

In [ ]:
print(maxlen_sent, maxlen_doc)

18 79


In [ ]:
def pad_sequences(seq, maxlen, pad_first=True, pad_idx=1):
    if len(seq) >= maxlen:
        res = np.array(seq[-maxlen:]).astype("int32")
        return res
    else:
        res = np.zeros(maxlen, dtype="int32") + pad_idx
        if pad_first:
            res[-len(seq) :] = seq
        else:
            res[: len(seq) :] = seq
        return res


def pad_nested_sequences(
    seq, maxlen_sent, maxlen_doc, pad_sent_first=True, pad_doc_first=False, pad_idx=1
):
    seq = [s for s in seq if len(s) >= 1]
    if len(seq) == 0:
        return np.array([[pad_idx] * maxlen_sent] * maxlen_doc).astype("int32")
    seq = [pad_sequences(s, maxlen_sent, pad_sent_first, pad_idx) for s in seq]
    if len(seq) >= maxlen_doc:
        return np.array(seq[:maxlen_doc])
    else:
        res = np.array([[pad_idx] * maxlen_sent] * maxlen_doc).astype("int32")
        if pad_doc_first:
            res[-len(seq) :] = seq
        else:
            res[: len(seq) :] = seq
        return res

In [ ]:
padded_texts = np.stack([pad_nested_sequences(r, maxlen_sent, maxlen_doc) for r in texts_numz], axis=0)


In [ ]:
padded_texts.shape


(8675, 79, 18)

In [ ]:
padded_texts[0]


array([[   1,    1,    1,    1, ...,   14,  146, 1283,    0],
       [   1,    1,    1,    1, ..., 1873, 2093, 8979,    9],
       [   1,    1,    1,    1, ...,   51,  115,   36,    9],
       [   1,    1,    1,    1, ...,  101,   17,  366,    9],
       ...,
       [   1,    1,    1,    1, ...,    1,    1,    1,    1],
       [   1,    1,    1,    1, ...,    1,    1,    1,    1],
       [   1,    1,    1,    1, ...,    1,    1,    1,    1],
       [   1,    1,    1,    1, ...,    1,    1,    1,    1]], dtype=int32)

# HAN implementation

In [ ]:

import numpy as np
import torch
import torch.nn.functional as F

from torch import nn

In [ ]:
class AttentionWithContext(nn.Module):
    def __init__(self, hidden_dim):
        super(AttentionWithContext, self).__init__()

        self.attn = nn.Linear(hidden_dim, hidden_dim)
        self.contx = nn.Linear(hidden_dim, 1, bias=False)

    def forward(self, inp):
        # The first expression in the attention mechanism is simply a linear layer that receives 
        # the output of the Word-GRU referred here as 'inp' and h_{it} in the paper
        u = torch.tanh_(self.attn(inp))
        # The second expression is...the same but without bias, wrapped up in a Softmax function
        a = F.softmax(self.contx(u), dim=1)
        # And finally, an element-wise multiplication taking advantage of Pytorch's broadcasting abilities 
        s = (a * inp).sum(1)
        # we will also return the normalized importance weights
        return a.permute(0, 2, 1), s

In [ ]:
class Attention(nn.Module):
    def __init__(self, hidden_dim, seq_len):
        super(Attention, self).__init__()

        self.hidden_dim = hidden_dim
        self.seq_len = seq_len
        self.weight = nn.Parameter(nn.init.kaiming_normal_(torch.Tensor(hidden_dim, 1)))
        self.bias = nn.Parameter(torch.zeros(seq_len))

    def forward(self, inp):
        # 1. Matrix Multiplication
        x = inp.contiguous().view(-1, self.hidden_dim)
        u = torch.tanh_(torch.mm(x, self.weight).view(-1, self.seq_len) + self.bias)
        # 2. Softmax on 'u_{it}' directly
        a = F.softmax(u, dim=1)
        # 3. Braodcasting and out
        s = (inp * torch.unsqueeze(a, 2)).sum(1)
        return a, s

In [ ]:
class WordAttnNet(nn.Module):
    def __init__(
        self,
        vocab_size,
        hidden_dim=32,
        padding_idx=1,
        embed_dim=50,
        embedding_matrix=None,
    ):
        super(WordAttnNet, self).__init__()

        if isinstance(embedding_matrix, np.ndarray):
            self.word_embed = nn.Embedding(
                vocab_size, embedding_matrix.shape[1], padding_idx=padding_idx
            )
            self.word_embed.weight = nn.Parameter(torch.Tensor(embedding_matrix))
            embed_dim = embedding_matrix.shape[1]
        else:
            self.word_embed = nn.Embedding(vocab_size, embed_dim, padding_idx=padding_idx)

        self.rnn = nn.GRU(embed_dim, hidden_dim, bidirectional=True, batch_first=True)

        self.word_attn = AttentionWithContext(hidden_dim * 2)

    def forward(self, X, h_n):
        embed = self.word_embed(X.long())
        h_t, h_n = self.rnn(embed, h_n)
        a, s = self.word_attn(h_t)
        return a, s.unsqueeze(1), h_n

In [ ]:
bsz = 16
maxlen_sent = maxlen_sent # 20 
hidden_dim  = 32    
embed_dim   = 100    
vocab_size  = len(vocab.stoi)
padding_idx = 1

# net
word_embed = nn.Embedding(vocab_size, embed_dim, padding_idx=padding_idx)
rnn = nn.GRU(embed_dim, hidden_dim, bidirectional=True, batch_first=True)
attn = nn.Linear(hidden_dim*2, hidden_dim*2)
contx = nn.Linear(hidden_dim*2, 1, bias=False)

# inputs
X = torch.from_numpy(np.random.choice(vocab_size, (bsz, maxlen_sent)))
h_n = torch.zeros((2, bsz, hidden_dim))

In [ ]:
# 1. Word Embeddings
# (bsz, maxlen_sent, embed_dim)
embed = word_embed(X)
embed.shape

torch.Size([16, 18, 100])

In [ ]:

# 2. GRU
h_t, h_n = rnn(embed, h_n)
# (bsz, seq_len, hidden_dim*2)
h_t.shape

torch.Size([16, 18, 64])

In [ ]:

# 3. Attention
u = torch.tanh_(attn(h_t))
a = F.softmax(contx(u), dim=1)
print(h_t.shape, a.shape)

torch.Size([16, 18, 64]) torch.Size([16, 18, 1])


In [ ]:
# RNN outputs scaled by their importance weights
s = (a * h_t)
print(s.shape)
# Sum along the seq dim so we end up with a representation per document/review
s = s.sum(1)
print(s.shape)
# Because this will be stack for all sentences, we do the `.unsqueeze(1)`
print(s.unsqueeze(1).shape)

torch.Size([16, 18, 64])
torch.Size([16, 64])
torch.Size([16, 1, 64])


In [ ]:
class SentAttnNet(nn.Module):
    def __init__(
        self, word_hidden_dim=32, sent_hidden_dim=32, padding_idx=1
    ):
        super(SentAttnNet, self).__init__()

        self.rnn = nn.GRU(
            word_hidden_dim * 2, sent_hidden_dim, bidirectional=True, batch_first=True
        )

        self.sent_attn = AttentionWithContext(sent_hidden_dim * 2)

    def forward(self, X):
        h_t, h_n = self.rnn(X)
        a, v = self.sent_attn(h_t)
        return a.permute(0,2,1), v

In [ ]:
class HierAttnNet(nn.Module):
    def __init__(
        self,
        vocab_size,
        maxlen_sent,
        maxlen_doc,
        word_hidden_dim=32,
        sent_hidden_dim=32,
        padding_idx=1,
        embed_dim=50,
        embedding_matrix=None,
        num_class=4,
    ):
        super(HierAttnNet, self).__init__()

        self.word_hidden_dim = word_hidden_dim

        self.wordattnnet = WordAttnNet(
            vocab_size=vocab_size,
            hidden_dim=word_hidden_dim,
            padding_idx=padding_idx,
            embed_dim=embed_dim,
            embedding_matrix=embedding_matrix,
        )

        self.sentattnnet = SentAttnNet(
            word_hidden_dim=word_hidden_dim,
            sent_hidden_dim=sent_hidden_dim,
            padding_idx=padding_idx,
        )

        self.fc = nn.Linear(sent_hidden_dim * 2, num_class)

    def forward(self, X):
        x = X.permute(1, 0, 2)
        word_h_n = nn.init.zeros_(torch.Tensor(2, X.shape[0], self.word_hidden_dim))
        if use_cuda:
            word_h_n = word_h_n.cuda()
        # alpha and s Tensor Lists
        word_a_list, word_s_list = [], []
        for sent in x:
            word_a, word_s, word_h_n = self.wordattnnet(sent, word_h_n)
            word_a_list.append(word_a)
            word_s_list.append(word_s)
        # Importance attention weights per word in sentence
        self.sent_a = torch.cat(word_a_list, 1)
        # Sentences representation
        sent_s = torch.cat(word_s_list, 1)
        # Importance attention weights per sentence in doc and document representation
        self.doc_a, doc_s = self.sentattnnet(sent_s)
        return self.fc(doc_s)

In [ ]:
maxlen_sent = maxlen_sent # 20
maxlen_doc =  maxlen_doc # 5
num_class = len(cleaned_data['category'].unique()) # 4
word_hidden_dim = 32
sent_hidden_dim = 32

wordattnnet = WordAttnNet(vocab_size, hidden_dim, padding_idx, embed_dim, embedding_matrix=None)
sentattnnet = SentAttnNet(word_hidden_dim, sent_hidden_dim, padding_idx)
fc = nn.Linear(sent_hidden_dim * 2, num_class)

In [ ]:
X = torch.from_numpy(np.random.choice(vocab_size, (bsz, maxlen_doc, maxlen_sent)))


In [ ]:
x = X.permute(1, 0, 2)
x.shape

torch.Size([79, 16, 18])

In [ ]:
# Initial Word RNN hidden state
word_h_n = nn.init.zeros_(torch.Tensor(2, X.shape[0], word_hidden_dim))

In [ ]:
# Loop through sentences:
word_a_list, word_s_list = [], []
for sent in x:
    word_a, word_s, word_h_n = wordattnnet(sent, word_h_n)
    word_a_list.append(word_a)
    word_s_list.append(word_s)
# Importance attention weights per word in sentence
sent_a = torch.cat(word_a_list, 1)
# Sentences representation
sent_s = torch.cat(word_s_list, 1)
# (bsz, maxlen_doc, maxlen_sent)
print(sent_a.shape)
# (bsz, maxlen_doc, hidden_dim*2)
print(sent_s.shape)

torch.Size([16, 79, 18])
torch.Size([16, 79, 64])


In [ ]:

doc_a, doc_s = sentattnnet(sent_s)
# (bsz, maxlen_doc, 1). One could .squeeze(2)
print(doc_a.shape)
# (bsz, hidden_dim*2)
print(doc_s.shape)

torch.Size([16, 79, 1])
torch.Size([16, 64])


In [ ]:
out = fc(doc_s)
out.shape

torch.Size([16, 16])

# Utilities

In [ ]:
class CategoricalAccuracy(object):
    def __init__(self, top_k=1):
        self.top_k = top_k
        self.correct_count = 0
        self.total_count = 0

        self._name = "acc"

    def reset(self):
        self.correct_count = 0
        self.total_count = 0

    def __call__(self, y_pred, y_true):
        top_k = y_pred.topk(self.top_k, 1)[1]
        true_k = y_true.view(len(y_true), 1).expand_as(top_k)
        self.correct_count += top_k.eq(true_k).float().sum().item()
        self.total_count += len(y_pred)
        accuracy = float(self.correct_count) / float(self.total_count)
        return np.round(accuracy, 4)

# Preparing Data

In [ ]:
 X_train, X_valid, y_train, y_valid = train_test_split(
        padded_texts, cleaned_data.category_class, train_size=0.8, random_state=1, stratify=cleaned_data.category_class
    )

In [ ]:
X_valid, X_test, y_valid, y_test = train_test_split(
        X_valid, y_valid, train_size=0.5, random_state=1, stratify=y_valid
    )

# Running

In [ ]:
import pandas as pd
import numpy as np
import pickle
import torch
import os
import torch.nn.functional as F

from pathlib import Path
from tqdm import trange
from sklearn.metrics import accuracy_score, f1_score, precision_score
from torch.optim.lr_scheduler import CyclicLR, ReduceLROnPlateau
from torch.utils.data import TensorDataset, DataLoader


# from utils.metrics import CategoricalAccuracy
# from utils.parser import parse_args

In [ ]:
n_cpus = os.cpu_count()
use_cuda = torch.cuda.is_available()

In [ ]:
# this wont be used

data_dir = Path("data")
train_dir = data_dir / "train"
valid_dir = data_dir / "valid"
test_dir = data_dir / "test"

ftrain, fvalid, ftest = "han_train.npz", "han_valid.npz", "han_test.npz"
tokf = "HANPreprocessor.p"

In [ ]:
batch_size = 64

train_set = TensorDataset(
    torch.from_numpy(np.array(X_train)),
    torch.from_numpy(np.array(y_train)).long(),
)
train_loader = DataLoader(dataset=train_set, batch_size=batch_size, num_workers=n_cpus)


np.random.seed(2)

eval_set = TensorDataset(
    torch.from_numpy(np.array(X_valid)),
    torch.from_numpy(np.array(y_valid)).long(),
)
eval_loader = DataLoader(
    dataset=eval_set, batch_size=batch_size, num_workers=n_cpus, shuffle=False
)

In [ ]:
next(iter(train_loader))


[tensor([[[   1,    1,    1,  ...,    5,  145,    9],
          [   5,   18,   99,  ...,   29,   50,    9],
          [2932,  116,  144,  ...,   13,  308,  190],
          ...,
          [   1,    1,    1,  ...,    1,    1,    1],
          [   1,    1,    1,  ...,    1,    1,    1],
          [   1,    1,    1,  ...,    1,    1,    1]],
 
         [[ 216,   17,   18,  ...,  878,   34,    9],
          [   1,   10,  954,  ...,   26,  314,    9],
          [ 247,   47,  316,  ..., 2660,  677,    9],
          ...,
          [   1,    1,    1,  ...,    1,    1,    1],
          [   1,    1,    1,  ...,    1,    1,    1],
          [   1,    1,    1,  ...,    1,    1,    1]],
 
         [[   1,    1,   65,  ...,   31,  353,    9],
          [  34,  173, 3134,  ...,   37,   31,    9],
          [   1,    5,   18,  ...,   12, 1040, 6465],
          ...,
          [   1,    1,    1,  ...,    1,    1,    1],
          [   1,    1,    1,  ...,    1,    1,    1],
          [   1,    1,    1,  .

In [ ]:
model = HierAttnNet(
    vocab_size=len(vocab.stoi),
    maxlen_sent=maxlen_sent,
    maxlen_doc=maxlen_doc,
    word_hidden_dim=32,
    sent_hidden_dim=32,
    padding_idx=1,
    embed_dim=50,
    # weight_drop=0.,
    # embed_drop=0.,
    # locked_drop=0.,
    # last_drop=0.,
    embedding_matrix=None,
    num_class=len(cleaned_data['category'].unique()),
)

In [ ]:
model

HierAttnNet(
  (wordattnnet): WordAttnNet(
    (word_embed): Embedding(70082, 50, padding_idx=1)
    (rnn): GRU(50, 32, batch_first=True, bidirectional=True)
    (word_attn): AttentionWithContext(
      (attn): Linear(in_features=64, out_features=64, bias=True)
      (contx): Linear(in_features=64, out_features=1, bias=False)
    )
  )
  (sentattnnet): SentAttnNet(
    (rnn): GRU(64, 32, batch_first=True, bidirectional=True)
    (sent_attn): AttentionWithContext(
      (attn): Linear(in_features=64, out_features=64, bias=True)
      (contx): Linear(in_features=64, out_features=1, bias=False)
    )
  )
  (fc): Linear(in_features=64, out_features=16, bias=True)
)

# Training

In [ ]:
optimizer = torch.optim.AdamW(model.parameters())
# This class is at the utils module
metric = CategoricalAccuracy()

In [ ]:
def train_step(model, optimizer, train_loader, epoch, metric):
    model.train()
    metric.reset()
    train_steps = len(train_loader)
    running_loss = 0
    with trange(train_steps) as t:
        for batch_idx, (data, target) in zip(t, train_loader):
            t.set_description("epoch %i" % (epoch + 1))

            X = data.cuda() if use_cuda else data
            y = target.cuda() if use_cuda else target

            optimizer.zero_grad()
            y_pred = model(X)
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            avg_loss = running_loss / (batch_idx + 1)
            acc = metric(F.softmax(y_pred, dim=1), y)

            t.set_postfix(acc=acc, loss=avg_loss)


def eval_step(model, eval_loader, metric, is_test=False):
    model.eval()
    metric.reset()
    eval_steps = len(eval_loader)
    running_loss = 0
    preds = []
    with torch.no_grad():
        with trange(eval_steps) as t:
            for batch_idx, (data, target) in zip(t, eval_loader):
                if is_test:
                    t.set_description("test")
                else:
                    t.set_description("valid")

                X = data.cuda() if use_cuda else data
                y = target.cuda() if use_cuda else target

                y_pred = model(X)
                loss = F.cross_entropy(y_pred, y)
                running_loss += loss.item()
                avg_loss = running_loss / (batch_idx + 1)
                acc = metric(F.softmax(y_pred, dim=1), y)
                if is_test:
                    preds.append(y_pred)
                t.set_postfix(acc=acc, loss=avg_loss)

    return avg_loss, preds


def early_stopping(curr_value, best_value, stop_step, patience):
    if curr_value <= best_value:
        stop_step, best_value = 0, curr_value
    else:
        stop_step += 1
    if stop_step >= patience:
        print("Early stopping triggered. log:{}".format(best_value))
        stop = True
    else:
        stop = False
    return best_value, stop_step, stop

In [ ]:
metric = CategoricalAccuracy()
n_epochs = 23
eval_every = 1
patience = 3 # 1
stop_step = 0
best_loss = 1e6
for epoch in range(n_epochs):
    train_step(model, optimizer, train_loader, epoch, metric)
    if epoch % eval_every == (eval_every - 1):
        val_loss, _ = eval_step(model, eval_loader, metric)
        best_loss, stop_step, stop = early_stopping(
            val_loss, best_loss, stop_step, patience
        )
    if stop:
        break

valid: 100%|██████████| 14/14 [00:05<00:00,  2.51it/s, acc=0.561, loss=1.65]

Early stopping triggered. log:1.4968211821147375


# Running on custom inputs

In [ ]:
def find_personality(sentence, clean=False):
  # clean the sentence if required
  if clean:
    sentence = sentence.strip()
    # Step 1: Remove links
    sentence = re.sub(r'http\S+', '', sentence)
    # Step 2: Remove multiple full stops
    sentence = re.sub(r'\.+', ".", sentence)
    # Step 3: Some sentences do not have space after full stop, remove that
    sentence = sentence.replace(".", ". ")
    # Step 4: remove multiple spaces
    sentence = re.sub(' +', ' ', sentence)
    # Step 5: remove punctuations except . , ! & ? '
    sentence = re.sub("[^a-zA-Z.,!&?']", " ", sentence)
  
  # convert sentence into list of sentence
  sentences = [sentence]


  tok_func = spacy.load("en_core_web_sm")
  n_cpus = os.cpu_count()
  texts_sents = []
  for doc in tok_func.pipe(sentences, n_process=n_cpus):
      sents = [str(s) for s in list(doc.sents)]
      texts_sents.append(sents)

  all_sents = [s for sents in texts_sents for s in sents]
  texts_length = [0] + [len(s) for s in texts_sents]
  range_idx = [sum(texts_length[: i + 1]) for i in range(len(texts_length))]

  sents_tokens = Tokenizer().process_all(all_sents)

  sents_tokens_2 = get_texts(all_sents, with_preprocess=True)

  new_tok = Tokenizer()
  new_tok.pre_rules = [rm_punctuation] + new_tok.pre_rules  

  sents_numz = [vocab.numericalize(s) for s in sents_tokens]

  texts_numz = [sents_numz[range_idx[i] : range_idx[i + 1]] for i in range(len(range_idx[:-1]))]

  padded_texts = np.stack([pad_nested_sequences(r, maxlen_sent, maxlen_doc) for r in texts_numz], axis=0)

  custom_data = torch.from_numpy(np.array(padded_texts))

  X = custom_data.cuda() if use_cuda else custom_data
  y_pred = model(X)
  # indx = torch.max(y_pred[0])
  idx2 = int(torch.argmax(y_pred[0]).cpu().numpy())
  sent_category = categories[idx2]
  return sent_category


In [ ]:
find_personality("If you're visiting this page, you're likely here because you're searching for a random sentence. Sometimes a random word just isn't enough, and that is where the random sentence generator comes into play. By inputting the desired number, you can make a list of as many random sentences as you want or need. Producing random sentences can be helpful in a number of different ways.")

'INTP'